**Een Identifier die gebruik maakt van active learning. 500 vragen zijn zelf gelabeled. Vervolgens fine tunen we een bert model hierop. We laten deze getunede bert dan zelf 1000 ongeziene vragen labelen. We kijken naar de 20% waar het model het minst zeker van is. Die kijken we handmatig na. De gecorrigeerde voegen we dan toe aan de training data en zo herhalen we dit proces tot het bert model naar behoren werkt.** 

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load labeled data
df = pd.read_excel("Trainig_data.xlsx")  
df.columns = ['question', 'label']  

# Split into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [11]:
'''
Een random lange zin om te zien of we met 128 tokens per zin goed zitten. Deze zin bevat 44 dus 128 zou ok moeten zijn 
'''


tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')
tokenizer2 = BertTokenizer.from_pretrained("bert-base-multilingual-cased" )
tokenizer3 = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

text = "Zal de minister initiatieven nemen om ervoor te zorgen dat personen in een palliatief zorgtraject in de toekomst wel aanspraak kunnen maken op een tegemoetkoming van hulpmiddelen? Zo ja, welke en binnen welke termijn?"

tokens = tokenizer.tokenize(text)
tokens2 = tokenizer2.tokenize(text)
tokens3 = tokenizer3.tokenize(text)

print("Number of tokens:", len(tokens))
print("Tokens:", tokens)

print("Number of tokens:", len(tokens2))
print("Tokens:", tokens2)

print("Number of tokens:", len(tokens3))
print("Tokens:", tokens3)

Number of tokens: 44
Tokens: ['Zal', 'de', 'minister', 'initiatie', '##ven', 'nemen', 'om', 'ervoor', 'te', 'zorgen', 'dat', 'personen', 'in', 'een', 'pal', '##lia', '##tief', 'zorgt', '##ra', '##ject', 'in', 'de', 'toekomst', 'wel', 'aan', '##spraak', 'kunnen', 'maken', 'op', 'een', 'tegemoetkoming', 'van', 'hulpmiddel', '##en', '?', 'Zo', 'ja', ',', 'welke', 'en', 'binnen', 'welke', 'termijn', '?']
Number of tokens: 55
Tokens: ['Za', '##l', 'de', 'minister', 'init', '##iati', '##even', 'nemen', 'om', 'ervoor', 'te', 'zo', '##rgen', 'dat', 'personen', 'in', 'een', 'pal', '##lia', '##tief', 'zo', '##rgt', '##raj', '##ect', 'in', 'de', 'toe', '##komst', 'wel', 'aan', '##spraak', 'kunnen', 'maken', 'op', 'een', 'te', '##gem', '##oe', '##tkom', '##ing', 'van', 'hulp', '##mid', '##delen', '?', 'Zo', 'ja', ',', 'welke', 'en', 'binnen', 'welke', 'term', '##ijn', '?']
Number of tokens: 40
Tokens: ['Zal', 'Ġde', 'Ġminister', 'Ġinitiatieven', 'Ġnemen', 'Ġom', 'Ġervoor', 'Ġte', 'Ġzorgen', 'Ġdat'

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')  # or 'GroNLP/bert-base-dutch-cased'

def tokenize(texts):
    return tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt", max_length=128
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)


In [9]:

class QuestionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = QuestionDataset(train_encodings, train_labels)
val_dataset = QuestionDataset(val_encodings, val_labels)


In [10]:
model = BertForSequenceClassification.from_pretrained('GroNLP/bert-base-dutch-cased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.421758
2,No log,0.358749
3,0.370200,0.403158


TrainOutput(global_step=717, training_loss=0.3160602989888424, metrics={'train_runtime': 108.8975, 'train_samples_per_second': 52.563, 'train_steps_per_second': 6.584, 'total_flos': 376511920220160.0, 'train_loss': 0.3160602989888424, 'epoch': 3.0})

In [7]:
import torch
import torch.nn.functional as F
import pandas as pd

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load data
unlabeled_df = pd.read_excel("unlabeled_questions.xlsx")
questions = unlabeled_df['question'].tolist()
encodings = tokenize(questions)
encodings = {k: v.to(device) for k, v in encodings.items()}

# Run model on the data
with torch.no_grad():
    outputs = model(**encodings)
    probs = F.softmax(outputs.logits, dim=1)
    
    predicted_probs, predicted_labels = torch.max(probs, dim=1)
    uncertainty = 1 - predicted_probs  # lower confidence = more uncertain

# Add results to DataFrame
unlabeled_df['predicted_label'] = predicted_labels.cpu().numpy()
unlabeled_df['confidence'] = predicted_probs.cpu().numpy()
unlabeled_df['uncertainty'] = uncertainty.cpu().numpy()

# Sort by uncertainty and get top 20%
top_uncertain = unlabeled_df.sort_values(by='uncertainty', ascending=False).head(int(0.2 * len(unlabeled_df)))

# Save to Excel for manual labeling
top_uncertain.to_excel("to_label_eng.xlsx", index=False)



In [11]:
# Load test data
df_test = pd.read_excel("Testset.xlsx")
df_test.columns = ['question', 'label']

# Clean and convert to string
df_test = df_test.dropna(subset=['question', 'label'])
df_test['question'] = df_test['question'].astype(str)

# Extract questions and labels
test_texts = df_test['question'].tolist()
test_labels = df_test['label'].tolist()

# Tokenize
test_encodings = tokenizer(
    test_texts, padding=True, truncation=True, return_tensors="pt", max_length=128
)


In [12]:
from transformers import Trainer

test_dataset = QuestionDataset(test_encodings, test_labels)

# Assuming you already have a `trainer` object that was used for training
trainer.evaluate(test_dataset)


{'eval_loss': 0.28848540782928467,
 'eval_runtime': 1.9543,
 'eval_samples_per_second': 255.333,
 'eval_steps_per_second': 32.236,
 'epoch': 3.0}

In [13]:
import numpy as np
from sklearn.metrics import classification_report

# Get raw predictions
outputs = trainer.predict(test_dataset)
preds = np.argmax(outputs.predictions, axis=1)

# Print evaluation report
print(classification_report(test_labels, preds))


              precision    recall  f1-score   support

           0       0.96      0.94      0.95       362
           1       0.85      0.89      0.87       137

    accuracy                           0.93       499
   macro avg       0.91      0.92      0.91       499
weighted avg       0.93      0.93      0.93       499



In [15]:
import pandas as pd
import re
from sklearn.metrics import classification_report

# Load test data
df_test = pd.read_excel("Testset.xlsx")
df_test.columns = ['question', 'label']
df_test = df_test.dropna(subset=['question'])
df_test['question'] = df_test['question'].astype(str)

# Your regex sets
statistical_sets = {
    "Set 1": [
        r"\b(hoeveel|aantal|percentage van|percentage|cijfer over|data over|statistieken van)\b",
        r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van)?\s*(de|een)?\s*(gegevens|statistieken|cijfers)\b"
    ],
    "Set 2": [
        r"\b(hoe vaak|hoe groot|gemiddelde van|mediaan van|ratio van|procent van)\b",
        r"\b(stijging van|daling van|verandering in|ontwikkeling in|schommeling van|impact op)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|analyse van)?\s*(de|een)?\s*(cijfers|gegevens|statistieken|tellingen)\b"
    ],
    "Set 3": [
        r"\b(bruto|netto|inkomen|uitgaven|begroting|subsidies|belasting|tarief|BBP|inflatie|schulden|werkloosheid|bestedingen|consumptie)\b",
        r"\b(bevolking|leeftijdsgroep|demografie|migratie|emigratie|immigratie|huishoudens|gezinnen|verhouding tussen|dichtheid)\b",
        r"\b(aantal|hoeveelheid|grootte van|gemiddelde|mediaan|percentage|spreiding|percentiel|kwartiel|standaarddeviatie)\b"
    ],
    "Set 4": [
        r"\b(zorgkosten|patiënten|ziekenhuisopnames|sterftecijfers|levensverwachting|gezondheidsuitgaven|vaccinaties|epidemieën|medicatiegebruik)\b",
        r"\b(reistijd|filedruk|kilometers afgelegd|verkeersongevallen|CO2-uitstoot|luchtvervuiling|hernieuwbare energie|klimaatverandering|waterkwaliteit)\b",
        r"\b(grondprijzen|woningmarkt|huurprijzen|hypotheken|verkoopcijfers|bouwvergunningen|energieverbruik)\b"
    ],
    "Set 5": [
        r"\b(vergeleken met|ten opzichte van|in vergelijking met|in het verleden|sinds \d{4}|tussen \d{4} en \d{4})\b",
        r"\b(ontwikkeling sinds|historische gegevens|trendanalyse|jaarverslagen|statistische rapporten)\b"
    ],
    "Set 6": [
        r"\b(hoeveel|aantal|percentage|statistieken|cijfers|gegevens|data)\b",
        r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
        r"\b(per\s+\w+|per\s+\d+|in\s+\d{4}|tussen\s+\d{4}\s+en\s+\d{4})\b",  
        r"\b(wat is het aantal|hoe groot is|hoe vaak|gemiddelde van|ratio van)\b"
    ],
    "Set 7": [
        r"^(Wat is het aantal|Hoeveel)\b",
        r"\b(totale budget|welk budget|eenzelfde hoogte|gemiddeld aantal|geëvolueerd in de periode|jaarlijkse kostprijs |het aantal)\b",
        r"\b(bedroeg|welk|Kan de minister|wat was)\s*(de|een)?\s*(factuur|budget|overzicht|kostprijs)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|bedroeg)?\s*(de|een)?\s*(gegevens|statistieken|cijfers|factuur|overzicht)\b"
    ],
    "Set 8": [
        r"^(Wat is het aantal|Hoeveel)\b",
        r"\b(totale budget|welk budget|cijfer over|eenzelfde hoogte|gemiddeld aantal|geëvolueerd in de periode|jaarlijkse kostprijs|het aantal)\b",
        r"\b(bedroeg|welk|Kan de minister|wat was)\s*(de|een)?\s*(factuur|budget|overzicht|kostprijs)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|bedroeg)?\s*(de|een)?\s*(gegevens|statistieken|cijfers|factuur|overzicht)\b"
    ],
    "Set 9": [
        r"^(Hoeveel)\b"
    ]
}

# Function to apply regex rules to each question
def matches_any(question, patterns):
    return any(re.search(pat, question, flags=re.IGNORECASE) for pat in patterns)

# Evaluate each regex set
for set_name, patterns in statistical_sets.items():
    df_test[f"{set_name}_match"] = df_test['question'].apply(lambda q: int(matches_any(q, patterns)))

    if 'label' in df_test.columns:
        print(f"\n🧪 {set_name} — Evaluation against true labels:")
        print(classification_report(df_test['label'], df_test[f"{set_name}_match"], digits=3))



🧪 Set 1 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.914     0.914     0.914       362
           1      0.774     0.774     0.774       137

    accuracy                          0.876       499
   macro avg      0.844     0.844     0.844       499
weighted avg      0.876     0.876     0.876       499


🧪 Set 2 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.728     0.961     0.829       362
           1      0.333     0.051     0.089       137

    accuracy                          0.711       499
   macro avg      0.531     0.506     0.459       499
weighted avg      0.620     0.711     0.625       499


🧪 Set 3 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.750     0.953     0.839       362
           1      0.564     0.161     0.250       137

    accuracy                          0.735       499

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# Load data
df_train = pd.read_excel("Trainig_data.xlsx")
df_test = pd.read_excel("Testset.xlsx")
df_train.columns = df_test.columns = ['question', 'label']
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# Vectorize
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(df_train['question'])
y_train = df_train['label']
X_test = vectorizer.transform(df_test['question'])
y_test = df_test['label']

# Models with your parameters
models = {
    "Naive Bayes": MultinomialNB(alpha=0.01),
    "Logistic Regression": LogisticRegression(
        C=10, class_weight='balanced', penalty='l2', solver='saga', max_iter=1000
    ),
    "SVM": SVC(C=1, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(
        class_weight='balanced', max_depth=None, n_estimators=250
    ),
    "XGBoost": XGBClassifier(
        max_depth=4, learning_rate=0.3, n_estimators=250,
        use_label_encoder=False, eval_metric='logloss'
    )
}

# Train and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"\n🧪 {name}:\n")
    print(classification_report(y_test, preds, digits=3))



🧪 Naive Bayes:

              precision    recall  f1-score   support

           0      0.787     0.961     0.866       362
           1      0.754     0.314     0.443       137

    accuracy                          0.784       499
   macro avg      0.771     0.638     0.654       499
weighted avg      0.778     0.784     0.750       499


🧪 Logistic Regression:

              precision    recall  f1-score   support

           0      0.933     0.923     0.928       362
           1      0.801     0.825     0.813       137

    accuracy                          0.896       499
   macro avg      0.867     0.874     0.870       499
weighted avg      0.897     0.896     0.896       499


🧪 SVM:

              precision    recall  f1-score   support

           0      0.893     0.964     0.927       362
           1      0.880     0.693     0.776       137

    accuracy                          0.890       499
   macro avg      0.886     0.829     0.851       499
weighted avg      0.889

C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\xgboost\core.py:158: UserWarning: [14:23:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🧪 XGBoost:

              precision    recall  f1-score   support

           0      0.895     0.945     0.919       362
           1      0.829     0.708     0.764       137

    accuracy                          0.880       499
   macro avg      0.862     0.826     0.842       499
weighted avg      0.877     0.880     0.877       499

